# Лабораторная работа №1: Первичное исследование данных

## 1. Постановка задачи

### Описание датасета
Данный датасет содержит результаты регистрации сигналов в системе позиционирования, где взаимодействуют якоря (anchor) и метки (tag). Каждая строка отражает событие приёма сигнала с указанием идентификаторов устройств, служебных параметров и временных меток. Данные позволяют анализировать последовательность и частоту событий во времени.

### Условный заказчик
Инженерная команда или разработчики системы позиционирования, выполняющие анализ корректности работы оборудования и логов взаимодействия устройств.

### Возможные задачи ИАД
1. Выявление аномалий в работе системы по временным меткам и последовательностям сигналов.
2. Анализ закономерностей взаимодействия между якорями и метками.
3. Профилирование поведения системы во времени.

## 2. Паспорт датасета

### Загрузка данных

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('../data/blink.csv')

# Первая строка — реальные заголовки
df.columns = df.iloc[0]
df = df.drop(0)

print(f'Размер датасета: {df.shape[0]} строк, {df.shape[1]} столбцов')
df.head()

По результатам загрузки датасета видно, что данные изначально имеют некорректную структуру: первая строка используется как заголовки столбцов. Размер датасета составляет 609 строк и 8 столбцов, что соответствует требованиям лабораторной работы.

### Структура данных

In [ ]:
df.info()
df.describe()

Анализ структуры данных показывает, что большинство признаков имеют тип object, что не соответствует их реальному смыслу. Числовые и временные признаки представлены в виде строк, что затрудняет анализ и требует приведения типов.


### Приведение типов данных

In [ ]:
df['sequenceID'] = pd.to_numeric(df['sequenceID'], errors='coerce')
df['pan'] = pd.to_numeric(df['pan'], errors='coerce')
df['processed_flag'] = pd.to_numeric(df['processed_flag'], errors='coerce')
df['timestampToA'] = pd.to_numeric(df['timestampToA'], errors='coerce')
df['Timestamp ToA'] = pd.to_numeric(df['Timestamp ToA'], errors='coerce')

df['datetime'] = pd.to_datetime(df['Timestamp ToA'], unit='ms')

df['anchorID'] = df['anchorID'].astype('category')
df['tagID'] = df['tagID'].astype('category')

df.info()

После приведения типов данные стали пригодны для анализа. Были выделены числовые, категориальные и временные признаки. Это позволяет корректно выполнять статистический анализ и визуализацию.

## 3. Аудит качества данных

### 3.1. Пропуски

In [ ]:
missing = pd.DataFrame({
    'Пропуски': df.isnull().sum(),
    'Доля (%)': (df.isnull().sum() / len(df) * 100).round(2)
}).sort_values('Пропуски', ascending=False)

missing[missing['Пропуски'] > 0]

Наибольшее количество пропусков наблюдается в столбцах временных меток после преобразования типов. Это указывает на наличие некорректных значений в исходном файле. Пропуски в числовых столбцах критичны, так как могут повлиять на анализ последовательностей событий.

### 3.2. Дубликаты

In [ ]:
duplicates = df.duplicated().sum()
print(f'Полных дубликатов строк: {duplicates}')

Полных дубликатов строк не обнаружено. Это говорит о том, что записи являются уникальными событиями регистрации сигналов.

### 3.3. Выбросы (по признаку sequenceID)

In [ ]:
column = 'sequenceID'
Q1 = df[column].quantile(0.25)
Q3 = df[column].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
print(f'Выбросов по методу IQR: {len(outliers)}')

plt.figure(figsize=(10, 6))
sns.boxplot(data=df, y=column)
plt.title(f'Boxplot для {column}')
plt.show()

По графику boxplot видно наличие выбросов в значениях sequenceID. Это может указывать на сбои в последовательности регистрации сигналов или ошибки оборудования.

## 4. Разведочный анализ (EDA)

### 4.1. Распределение sequenceID

In [ ]:
numeric_col = 'sequenceID'

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
sns.histplot(df[numeric_col], kde=True)
plt.title(f'Распределение {numeric_col}')

plt.subplot(1, 2, 2)
sns.boxplot(y=df[numeric_col])
plt.title(f'Boxplot {numeric_col}')
plt.tight_layout()
plt.show()

Распределение sequenceID показывает неравномерность появления сигналов. Наличие длинного хвоста распределения подтверждает присутствие аномальных значений.

### 4.2. Анализ категориального признака anchorID

In [ ]:
cat_col = 'anchorID'

plt.figure(figsize=(10, 6))
top_categories = df[cat_col].value_counts().head(10)
sns.barplot(x=top_categories.values, y=top_categories.index)
plt.title(f'Топ-10 категорий в {cat_col}')
plt.xlabel('Количество')
plt.show()

Диаграмма частот показывает, что некоторые якоря используются значительно чаще других. Это может быть связано с их физическим расположением или особенностями работы системы.

## 5. Выводы

Детали анализа и выявленные проблемы качества данных приведены в файле `report/quality_report.md`.